In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [2]:
imp_df = pd.read_csv('imp_df_2.csv')

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
imp_df['category_id'] = imp_df['category_id'].astype(str)
category = [i[:3] for i in imp_df['category_id']]
imp_df['category'] = category

In [6]:
up_pids = pd.read_csv('up_pids.csv')
pid_day_diff = pd.read_csv('pid_created_day_diff.csv')

In [9]:
up_pids['ifUp'] = ['True'] * len(up_pids)

In [16]:
up_pids.columns = ['content_id', 'ifUp']

In [8]:
pid_day_diff.head()

,day_diff,content_id
0,565,99572032
1,557,100016640
2,557,100018176
3,565,99606848
4,564,99635264


In [17]:
imp_df = pd.merge(imp_df, up_pids, how='left', on='content_id')
imp_df = pd.merge(imp_df, pid_day_diff, on='content_id')

In [23]:
imp_df['ifUp'] = imp_df['ifUp'].fillna('False')

In [25]:
imp_df.head()

,day,imp_id,ref_term,user_id,content_id,content_position,content_owner,category_id,ad_type,ad_id,category,ifUp,day_diff
0,20201012,99815f83dbe419ebc463,노스페이스 핑크,12036541,76917587,101,150222,400050510,NaN,NaN,400,False,1053
1,20201012,97165f83332e17860967,노스페이스부츠,6642374,76917587,90,150222,400050510,NaN,NaN,400,False,1053
2,20201012,99815f83dc0219ec6b50,노스페이스 핑크,12036541,90792853,9,2805327,310090030,NaN,NaN,310,False,733
3,20201012,97165f83dbc719e4d24b,노스페이스 핑크,12036541,90792853,9,2805327,310090030,NaN,NaN,310,False,733
4,20201012,97165f83e1fc1a06fc00,핑크패딩,7173049,90792853,132,2805327,310090030,NaN,NaN,310,False,733


In [59]:
imp_df['ref_term'].nunique()

177422

In [43]:
keyword = imp_df.groupby('ref_term')['imp_id'].size().reset_index()
keyword.columns = ['ref_term', 'imp_cnt']

In [44]:
temp = keyword.groupby('imp_cnt')['ref_term'].count().reset_index()
temp = temp.sort_values('imp_cnt', ascending=False)
rank = np.array(range(1, len(temp)+1))
rank_reverse = np.array(range(len(temp), 0, -1))
temp['rank'] = rank
temp['rank_reverse'] = rank_reverse

In [45]:
temp = temp.drop('ref_term', axis = 1)

In [46]:
keyword = pd.merge(keyword, temp, how='left', on='imp_cnt')


,ref_term,imp_cnt,rank,rank_reverse
0,#0237,13,2449,13
1,#WH-1000XM4,5,2457,5
2,#app,1,2461,1
3,#java,11,2451,11
4,#과제,15,2447,15
5,#대학과제,3,2459,3
6,#롤,31,2431,31
7,#리퍼랜드,6,2456,6
8,#분당명품,4,2458,4
9,#브더사엽서,1,2461,1


In [47]:
keyword.sort_values('imp_cnt', ascending=False)

,ref_term,imp_cnt,rank,rank_reverse
135830,자전거,51679,1,2461
111669,아이폰,37245,2,2460
27772,골프,32881,3,2459
110703,아이즈원,32675,4,2458
101967,스톤아일랜드,32635,5,2457
34821,나이키,32054,6,2456
42921,노트북,31203,7,2455
124949,오토바이,31003,8,2454
169570,피규어,29724,9,2453
158553,톰브라운,27726,10,2452


In [50]:
keyword.columns = ['ref_term', 'keyword_imp_cnt', 'rank', 'rank_reverse']

12753031

In [66]:
imp_df = pd.merge(imp_df, keyword, on='ref_term')

In [52]:
imp_df.head()

,day,imp_id,ref_term,user_id,content_id,content_position,content_owner,category_id,ad_type,ad_id,category,ifUp,day_diff,keyword_imp_cnt,rank,rank_reverse
0,20201012,99815f83dbe419ebc463,노스페이스 핑크,12036541,76917587,101,150222,400050510,NaN,NaN,400,False,1053,35,2427,35
1,20201012,99815f83dc0219ec6b50,노스페이스 핑크,12036541,90792853,9,2805327,310090030,NaN,NaN,310,False,733,35,2427,35
2,20201012,97165f83dbc719e4d24b,노스페이스 핑크,12036541,90792853,9,2805327,310090030,NaN,NaN,310,False,733,35,2427,35
3,20201012,99815f83dc0219ec6b51,노스페이스 핑크,12036541,94139647,10,3357019,400050500,NaN,NaN,400,False,668,35,2427,35
4,20201012,97165f83dbc719e4d24c,노스페이스 핑크,12036541,94139647,10,3357019,400050500,NaN,NaN,400,False,668,35,2427,35


In [53]:
imp_df.to_csv('imp_df_2.csv', index=False)

In [55]:
ad = imp_df[~imp_df['ad_type'].isna()]
no_ad = imp_df[(imp_df['ad_type'].isna()) & (imp_df['ifUp'] == 'False')]
up = imp_df[(imp_df['ad_type'].isna()) & (imp_df['ifUp'] == 'True')]

In [56]:
ad_temp = ad.groupby(['content_id', 'ref_term', 'rank'])['imp_id'].size().reset_index()
ad_temp.columns = ['content_id', 'ref_term', 'rank', 'imp_cnt']

no_ad_temp = no_ad.groupby(['content_id', 'ref_term', 'rank'])['imp_id'].size().reset_index()
no_ad_temp.columns = ['content_id', 'ref_term', 'rank', 'imp_cnt']

up_temp = up.groupby(['content_id', 'ref_term', 'rank'])['imp_id'].size().reset_index()
up_temp.columns = ['content_id', 'ref_term', 'rank', 'imp_cnt']

In [57]:
cache = {}
cache['ad'] = ad
cache['no_ad'] = no_ad
cache['up'] = up
cache['ad_ref_term'] = ad_temp
cache['no_ad_ref_term'] = no_ad_temp
cache['up_ref_term'] = up_temp

In [58]:
import pickle

with open('cache.pickle', 'wb') as handle:
    pickle.dump(cache, handle, protocol=pickle.HIGHEST_PROTOCOL)